LibAUC Experiments

In [2]:
from libauc.models import resnet18
from libauc.losses import AUCMLoss

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator


from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import densenet121 as DenseNet121
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore") 

In [3]:
# data_flag = 'pathmnist'
data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 16
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [4]:
task

'binary-class'

In [28]:
# # preprocessing
# data_transform = transforms.Compose([
#     transforms.ToTensor(),
#     # transforms.Grayscale(3),
#     transforms.Resize((28, 28)), 
#     transforms.Normalize(mean=[.5], std=[.5])
# ])

data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            transforms.Resize((32, 32)), 
            
            transforms.Normalize(0.5, 0.5),
        ])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = torch_data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = torch_data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = torch_data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz


In [29]:
data[0].shape

torch.Size([16, 1, 32, 32])

## BEST CE Result for BMNIST below

In [36]:
from libauc.models import resnet18
from libauc.losses import AUCMLoss

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator


from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import densenet121 as DenseNet121
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore") 


data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 16
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

# Data Transformation
data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            transforms.Resize((32, 32)), 
            
            transforms.Normalize(0.5, 0.5),
        ])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = torch_data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = torch_data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = torch_data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)




# BEST CROSS ENTROPY 
lr = 1e-5
weight_decay = 2e-5
model = resnet18(num_classes=2)
model = model.cuda()
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# training
best_val_auc = 0 
for epoch in range(201):
    train_losses = []
    for idx, data in enumerate(train_loader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      loss = criterion(y_pred, train_labels.squeeze().type(torch.LongTensor).cuda())
      train_losses.append(loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch%5==0:
      print("Epoch : {:03d}  Train Loss : {:.5f}  ".format(epoch, np.mean(train_losses)), end='')
      model.eval()
      with torch.no_grad():    
          test_pred = []
          test_true = [] 
          test_losses = []
          for jdx, data in enumerate(test_loader):
              test_data, test_labels = data
              test_data = test_data.cuda()
              y_pred = model(test_data)
              test_pred.append(y_pred.cpu().detach().numpy())
              test_true.append(test_labels.numpy())
              test_losses.append(criterion(y_pred, test_labels.squeeze().type(torch.LongTensor).cuda()).item())

          test_true = np.concatenate(test_true)
          test_pred = np.concatenate(test_pred)
          val_auc_mean =  roc_auc_score(test_true.squeeze(), test_pred[:,1]) 
          print("Val Loss : {:.5f}   ".format(np.mean(test_losses)), end='')
          model.train()

          if best_val_auc < val_auc_mean:
              best_val_auc = val_auc_mean
              torch.save(model.state_dict(), 'ce_pretrained_model.pth')

          print ('BatchID= {}   Val_AUC={:.4f}   Best_Val_AUC={:.4f}'.format(
              idx, val_auc_mean, best_val_auc ))

Epoch : 000  Train Loss : 0.96551  Val Loss : 0.71291   BatchID= 34   Val_AUC=0.6009   Best_Val_AUC=0.6009
Epoch : 005  Train Loss : 0.46612  Val Loss : 0.53674   BatchID= 34   Val_AUC=0.7830   Best_Val_AUC=0.7830
Epoch : 010  Train Loss : 0.32923  Val Loss : 0.45366   BatchID= 34   Val_AUC=0.7993   Best_Val_AUC=0.7993
Epoch : 015  Train Loss : 0.27267  Val Loss : 0.44311   BatchID= 34   Val_AUC=0.8248   Best_Val_AUC=0.8248
Epoch : 020  Train Loss : 0.24289  Val Loss : 0.41883   BatchID= 34   Val_AUC=0.8331   Best_Val_AUC=0.8331
Epoch : 025  Train Loss : 0.20464  Val Loss : 0.41020   BatchID= 34   Val_AUC=0.8348   Best_Val_AUC=0.8348
Epoch : 030  Train Loss : 0.18576  Val Loss : 0.37117   BatchID= 34   Val_AUC=0.8707   Best_Val_AUC=0.8707
Epoch : 035  Train Loss : 0.16647  Val Loss : 0.36983   BatchID= 34   Val_AUC=0.8761   Best_Val_AUC=0.8761
Epoch : 040  Train Loss : 0.15724  Val Loss : 0.38295   BatchID= 34   Val_AUC=0.8860   Best_Val_AUC=0.8860
Epoch : 045  Train Loss : 0.13732  Va

In [12]:
# # # preprocessing
# # data_transform = transforms.Compose([
# #     transforms.ToTensor(),
# #     # transforms.Grayscale(3),
# #     transforms.Resize((28, 28)), 
# #     transforms.Normalize(mean=[.5], std=[.5])
# # ])

# train_transform = transforms.Compose([
#             transforms.ToTensor(),
#             transforms.ToPILImage(),
#             transforms.Grayscale(3),
#             transforms.ToTensor(),
#             transforms.Resize((32, 32)),  
#             transforms.RandomHorizontalFlip(0.3),
#             transforms.RandomVerticalFlip(0.3),
#             # transforms.RandomEqualize(0.3),    
#             # transforms.RandomErasing([p, scale, ratio, value, inplace]),
#             # transforms.AutoAugment(),               
#             # transforms.ElasticTransform(),
#             transforms.RandomRotation(30),
#             transforms.Normalize(0.5, 0.5),
#         ])

# test_transform = transforms.Compose([
#             transforms.ToTensor(),
#             transforms.ToPILImage(),
#             transforms.Grayscale(3),
#             transforms.ToTensor(),
#             transforms.Resize((32, 32)),             
#             transforms.Normalize(0.5, 0.5),
#         ])

# # load the data
# train_dataset = DataClass(split='train', transform=train_transform, download=download)
# test_dataset = DataClass(split='test', transform=data_transform, download=download)

# pil_dataset = DataClass(split='train', download=download)

# # encapsulate data into dataloader form
# train_loader = torch_data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# train_loader_at_eval = torch_data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
# test_loader = torch_data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


# lr = 0.01 # using smaller learning rate is better
# epoch_decay = 2e-5
# weight_decay = 1e-7
# margin = 1.0

# model = resnet18(num_classes=2)
# model = model.cuda()
# # criterion = nn.CrossEntropyLoss()  
# # optimizer = optim.Adam(model.parameters(), lr=lr)
# criterion = AUCMLoss()
# optimizer = PESG(model, 
#                  loss_fn=criterion, 
#                  lr=lr, 
#                  margin=margin, 
#                  epoch_decay=epoch_decay, 
#                  weight_decay=weight_decay)
# CE_loss = nn.CrossEntropyLoss()
# # training
# best_val_auc = 0 
for epoch in range(201):
    train_losses = []
    for idx, data in enumerate(train_loader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = criterion(y_pred, train_labels.squeeze().type(torch.LongTensor).cuda())
      train_losses.append(loss.item()/len(data))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
    #   if idx % 10 == 0:
    
    if epoch%5==0:
      print("Epoch : {:03d}  Train Loss : {:.5f} ".format(epoch, np.mean(train_losses)), end='')
      model.eval()
      with torch.no_grad():    
          test_pred = []
          test_true = [] 
          test_losses = []
          test_CE_losses = []
          for jdx, data in enumerate(test_loader):
              test_data, test_labels = data
              test_data = test_data.cuda()
              y_pred = model(test_data)
              test_pred.append(y_pred.cpu().detach().numpy())
              test_true.append(test_labels.numpy())
              test_losses.append(criterion(y_pred, test_labels.squeeze().type(torch.LongTensor).cuda()).item() / len(data))
              test_CE_losses.append(CE_loss(y_pred, test_labels.squeeze().type(torch.LongTensor).cuda()).cpu())

          test_true = np.concatenate(test_true)
          test_pred = np.concatenate(test_pred)
          val_auc_mean =  roc_auc_score(test_true.squeeze(), test_pred[:,1]) 
          print("Val Loss : {:.5f}   Val CE Loss : {:.5f}  ".format(np.mean(test_losses), np.mean(test_CE_losses)), end = '')
          model.train()

          if best_val_auc < val_auc_mean:
              best_val_auc = val_auc_mean
              torch.save(model.state_dict(), 'ce_pretrained_model.pth')

          print ('BatchID= {}   Val_AUC={:.4f}   Best_Val_AUC={:.4f}'.format(
              idx, val_auc_mean, best_val_auc ))

Epoch : 000  Train Loss : 0.05986 Val Loss : 1.08475   Val CE Loss : 0.66030  BatchID= 34   Val_AUC=0.7715   Best_Val_AUC=0.8079
Epoch : 005  Train Loss : 0.05511 Val Loss : 1.30268   Val CE Loss : 0.67616  BatchID= 34   Val_AUC=0.7575   Best_Val_AUC=0.8079
Epoch : 010  Train Loss : 0.05451 Val Loss : 0.95586   Val CE Loss : 0.69618  BatchID= 34   Val_AUC=0.7594   Best_Val_AUC=0.8079
Epoch : 015  Train Loss : 0.05996 Val Loss : 0.80668   Val CE Loss : 0.69962  BatchID= 34   Val_AUC=0.7907   Best_Val_AUC=0.8079
Epoch : 020  Train Loss : 0.06183 Val Loss : 1.06070   Val CE Loss : 0.73022  BatchID= 34   Val_AUC=0.7680   Best_Val_AUC=0.8079
Epoch : 025  Train Loss : 0.05332 Val Loss : 0.94242   Val CE Loss : 0.67984  BatchID= 34   Val_AUC=0.7701   Best_Val_AUC=0.8079
Epoch : 030  Train Loss : 0.05779 Val Loss : 1.07961   Val CE Loss : 0.67343  BatchID= 34   Val_AUC=0.7650   Best_Val_AUC=0.8079
Epoch : 035  Train Loss : 0.06424 Val Loss : 0.26654   Val CE Loss : 0.68599  BatchID= 34   Val_A

In [147]:
l1 =nn.CrossEntropyLoss()
l1(y_pred, test_labels.squeeze().type(torch.LongTensor).cuda())

tensor(0.7321, device='cuda:0')

In [145]:
test_labels.squeeze().type(torch.LongTensor).cuda()

tensor([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 1, 1, 1, 1, 1], device='cuda:0')

In [146]:
y_pred

tensor([[-2.1083, -2.4466],
        [-1.4856, -1.3008],
        [-0.0136, -0.7149],
        [-1.7003, -1.8327],
        [-1.8125, -1.2051],
        [-1.6341, -1.0869],
        [-2.0423, -2.1351],
        [-1.6785, -1.3222],
        [-1.9418, -1.4862],
        [-0.7217, -0.9000],
        [-2.1665, -1.4439],
        [-1.4988, -1.8446],
        [-1.3270, -1.8724],
        [-1.4483, -1.5633],
        [-0.6035, -1.0628],
        [-1.3239, -1.2100],
        [-1.4539, -1.3997],
        [-1.9640, -1.4416],
        [-1.2585, -1.1549],
        [-1.2373, -1.3626],
        [-2.0416, -2.3888],
        [-1.2686, -1.4588],
        [-0.5264, -0.5442],
        [-1.4517, -1.4630],
        [-2.0114, -1.4631],
        [-1.6009, -1.5631],
        [-1.7191, -1.2959],
        [-1.4363, -1.4959],
        [-1.4650, -1.6425],
        [-0.0229, -0.5606],
        [-1.1428, -1.1456],
        [-1.2979, -1.2396]], device='cuda:0')

In [1]:
target

NameError: name 'target' is not defined

In [22]:
model.layer1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)